# Coursework: The Museum Protection Problem

In [1]:
%load_ext autotime

time: 0 ns (started: 2023-12-07 03:56:07 +00:00)


update: Crossover = two point crossover

Result: 
    max fitness found is 2008.0 at gen 10 
time: 15.8 s

## Import relevant libraries

In [2]:
import numpy as np
import subprocess
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
import array
import csv


# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

time: 985 ms (started: 2023-12-07 03:56:07 +00:00)


## Set up  

Depending on the instance you are trying to solve, sest the following parameters


In [3]:
myinst="Instance/WallsTest1_5_cameras.csv"
instance_file = myinst  
nb_cameras = 5 
instance_size  = 100 
num_cells = instance_size*instance_size 


time: 0 ns (started: 2023-12-07 03:56:08 +00:00)


In [4]:
walls = np.zeros(instance_size*instance_size)

with open(myinst) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for line in csv_reader:
        column=int(line[0])
        row=int(line[1])
        oneD_index = (row * instance_size) + column;
        walls[oneD_index]=1
        

time: 0 ns (started: 2023-12-07 03:56:08 +00:00)


## Set up the fitness calculations 

This calls an external executable to return a fitness value. You cannot modify the code in this executable. 


1. If you are running code using **Google Colab**, you neyou must upload the executables called "bit_cam_napier_colab"  and "bit_cam_napier_visualisation_colab" to Colab and uncomment the line of code at tge top of the next cell

2. If you are running on **Windows using Anaconda**, then your code needs to call the executables "bit_cam_napier_windows.exe" and "bit_cam_napier_visualisation_windows.exe" - you need to specify the exact path to these on your system in the code below

3. If you are running on **Mac using Anaconda**, then your code needs to call the executables "bit_cam_napier_mac.exe" and "bit_cam_napier_visualisation_mac.exe" - you need to specify the exact path to these on your system in the code below

The call to the external function is wrapped inside an evaluation function called **eval_function()** .. this has the standard form used by DEAP and should be registered with the toolbox as usual. Read the following carefully.    You can any code you wish inside this function but it must always call the external *objective_function()* with a valid solution. This is defined below


 - The external "objective_function" needs to be passed a list of length (where n is the number of cells) where each value is 0 or 1.
- There must be exactly c values set to 1, where c= number of cameras
- Regardless of the representation you use, **you must turn a DEAP individual into a list of the form specified above** before calling the objective_function


**WARNING!!! You should only call the *objective_function* with a list that has exactly *c* bits set to 1**

If this is not the case, then you should assign a fitness value to the individual without calling the external  *objective_function*. You can choose how to set this - the fitness you assign should reflect the fact that the solution is invalid.  For a valid solution, the worst possible fitness is equal to *num_cells*, i.e. no cell is covered. Therefore, the fitness you choose to assign to an invalid solution should be worse than this value

In [5]:
path_binary = "CourseworkExecutables/bit_cam_napier_windows.exe"
path_binary_vis = "CourseworkExecutables/bit_cam_napier_visualisation_windows.exe"

# Do NOT modify this code - this calls an external binary with a solution
def objective_function(x, instance_size, nb_cameras, instance_file):
    params = ['%.16g' % a for a in x]
    cmd = [path_binary,str(instance_size),str(nb_cameras)]+params+[instance_file]
    s = subprocess.check_output(cmd)
    return float(s)

# Do NOT modify: this checks whether a camera is positioned on top of wall in a solution
def check_walls(solution, inst):
    clashes=0
    for i in range(0, len(solution)):
        if (walls[i] == 1 and solution[i]==1):
            clashes+=1
            
    return(clashes)        


time: 0 ns (started: 2023-12-07 03:56:08 +00:00)


In [6]:
# get the camera location (individual = location of cameras)
# change to bitstring in the size of num_cells
def convert_individual_bitstring(individual):
    converted_individual = np.zeros(num_cells)
    for i in range(len(individual)):
        converted_individual[individual[i]] = 1
    return converted_individual


time: 0 ns (started: 2023-12-07 03:56:08 +00:00)


In [7]:
fitness_cache = {}
def eval_function(individual):

    if (indiv_key:=tuple(individual)) in fitness_cache:
        return fitness_cache[indiv_key],
    
    solution=np.zeros(num_cells)
    solution = convert_individual_bitstring(individual)
    total_cameras = np.sum(solution)
    
    
    if instance_file == "":
        cameras_on_walls = 0
    else:
        cameras_on_walls = check_walls(solution, instance_file) 

    fitness = 0
    if  total_cameras > nb_cameras: # you decide!!!!
        fitness += num_cells * 10 * (total_cameras - nb_cameras)
    elif total_cameras < nb_cameras: # you decide!!
        fitness += num_cells * 2 * (nb_cameras - total_cameras)
        
    if cameras_on_walls > 0: # you decide!
        fitness += num_cells * cameras_on_walls

    if not fitness:
        # only call this if the solution contains exactly nb_cameras and no cameras on walls,
        fitness = objective_function(solution, instance_size, nb_cameras, instance_file)

    fitness_cache[indiv_key] = fitness
    return fitness,

time: 0 ns (started: 2023-12-07 03:56:08 +00:00)


## Set up the DEAP toolbox

This sets up a  very basic EA. It uses standard operators to manipulate a list of 0,1s.  You can change these operators for others from the DEAP toolbox or write your own

In [8]:
# Define the fitness class and create an individual class - set up as a minimization problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Create a toolbox
toolbox = base.Toolbox()

# Register 'attr_int' to generate a list of unique integers
toolbox.register("attr_int", random.sample, range(num_cells), nb_cameras)

# Register 'individual' to create an individual
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_int)

# Register 'population' to create a population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register all operators needed with the toolbox
toolbox.register("evaluate", eval_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.3)
toolbox.register("select", tools.selTournament, tournsize=2)


time: 0 ns (started: 2023-12-07 03:56:08 +00:00)


In [9]:
# main function - this just uses a simple EA - you can modify as you wish, including xhanging the parameters

ngen = 200
popsize = 50

def main():
    # Choose a population size: e.g. 50
    pop = toolbox.population(n=popsize)

    # Keep track of the single best solution found
    hof = tools.HallOfFame(1)

    # Create a statistics object
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Create a logbook to keep track of statistics
    logbook = tools.Logbook()

    # Percentage of population to replace
    PCT_REPLACE = 0.7

    # Run the algorithm
    for gen in range(ngen):
        offspring = algorithms.varAnd(pop, toolbox, cxpb=1.0, mutpb=0.05)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        pop = toolbox.select(offspring, int(len(pop)*(1-PCT_REPLACE)))
        pop += toolbox.population(n=int(len(pop)*PCT_REPLACE))

        # Update the statistics and record them in the logbook
        record = stats.compile(pop)
        logbook.record(gen=gen, **record)


    return pop, logbook, hof


time: 0 ns (started: 2023-12-07 03:56:08 +00:00)


# Experimenting

This runs the basic EA once. You can adapt this to run repeated expts. etc.

In [10]:

##############################
# run the main function
pop, log, hof = main()

##############################


best = hof[0].fitness.values[0]   # best fitness found is stored at index 0 in the hof list


# look in the logbook to see what generation this was found at

min = log.select("min")  # min fitness per generation stored in log

for i in range(ngen):  # set to ngen
        fit = min[i]
        if fit == best:
            break

print("max fitness found is %s at gen %s " % (best, i))
# print("number of cameras is %s" % (np.sum(hof[0])))




ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (25,) + inhomogeneous part.

time: 1.83 s (started: 2023-12-07 03:56:08 +00:00)


In [ ]:
import winsound
freq=1000
dur=2000
winsound.Beep(freq,dur)

In [ ]:
print("number of cameras is %s" % (len(hof[0])))

In [ ]:
print(hof[0])

In [ ]:
min

In [ ]:
len(pop)

In [ ]:
for i in pop:
    print(i.count(1))

In [ ]:
for i in hof:
    print(i)

## Plotting (optional)

In [ ]:
# This is not necessary - but useful to see what is going on for a single run
# code for plotting

gen = log.select("gen")
fit_max = log.select("max")
fit_min = log.select("min")
fit_avg = log.select("avg")

fig, ax1 = plt.subplots()

# set this to the number of generations in the EA
my_gens=ngen
# add a line to indicate "valid solutions" i.e solutions with the correct number of cameras. Any solution below this line is valid
worst_valid = [num_cells]*(my_gens+1)

line1 = ax1.plot(gen, fit_max, "r-", label="max Fitness")
line2 = ax1.plot(gen, fit_min, "b-", label="min Fitness")
line3 = ax1.plot(gen , fit_avg, "g-", label="avg Fitness")
line4 = ax1.plot(gen, worst_valid, "y-", label="worst valid soluton")
ax1.set_xlabel("Generations")
ax1.set_ylabel("Fitness", color="b")
for tl in ax1.get_yticklabels():
    tl.set_color("b")
    
#plt.ylim(0,num_cells+10)   # change the scale on the graph to see the valid solutions better

lns = line1+line2+line3+line4
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc="center right")



# Visualisation Tools

The code sets up some functions to allow you do visualise solutions. You shouldn't need to alter the code in the cell below (make sure the variable *path_binary_vis* is set to the correct binary at the start)



In [ ]:

# Run this cell to define the functions
def solution_converter(index, instance_size):
    counter = 0
    for i in range(instance_size):
        for j in range(instance_size):
            if (counter == index):
                return i,j
            else:
                counter+=1

# assumes path_binary is already set 
def visualise_objective_function(x, instance_size, nb_cameras, instance_file):
    params = ['%.16g' % a for a in x]
    cmd = [path_binary_vis,str(instance_size),str(nb_cameras)]+params+[instance_file]
    with open("temp_results.csv",'w') as fd:
        s = subprocess.call(cmd, stdout=fd)
    # Plotting the probability of detection
    data = pd.read_csv("temp_results.csv")
    values = np.array(data["value"])
    values[values < 0.5] = 0
    values = values.reshape(instance_size,instance_size)
    plt.imshow(values,cmap="Reds")
    # Plotting cameras
    cameras_indices = [index for (index, item) in enumerate(list(x)) if item == 1]
    for c in cameras_indices:
        x,y = solution_converter(c, instance_size=instance_size)
        plt.scatter(y,x,c="blue",marker="D")
    # Plotting walls (if any)
    if (instance_file != ""):
        walls = pd.read_csv(instance_file,header=None)
        plt.scatter(walls[1],walls[0],c="black", marker="_")
    plt.xlim(0,instance_size-1)
    plt.xlabel("y")
    plt.ylim(0,instance_size-1)
    plt.ylabel("x")
    plt.show()
    


### Visualising a solution

Two examples are given:
- visualising a random solution
- visualising an invidual returned from the EA


In [ ]:
# Here we just create a random solution for a 100x100 grid with two cameras at random positions
# you can experiment to see what different solutions look like


# this assumes a grid of size 100x100 
sol = np.zeros(100*100)

# add some cameras!
sol[random.randint(0, 10000)] = 1
sol[random.randint(0, 10000)] = 1
sol[random.randint(0, 10000)] = 1
sol[random.randint(0, 10000)] = 1
sol[random.randint(0, 10000)] = 1

actual_cameras=sum(sol)  # need to pass the actual numbers of cameras in the solution, regardless of whether it is valid

# choose an empty grid
#visualise_objective_function(sol, instance_size= 100, nb_cameras=2, instance_file="")

# choose a layout


visualise_objective_function(sol, instance_size=instance_size, nb_cameras=actual_cameras, instance_file=myinst)


In [ ]:
# # here we visualise the best result returned from the algorithm - it assumes an individual is a list of 0,1s. 
# # If your individual uses a different representation then adapt accordingly to reshape into this form

# # first we need to turn the individual returned into a list
# best_solution=[]
# for i in range(0, len(hof[0])):
#     best_solution.append(hof[0][i])
    
# actual_cameras=sum(best_solution)  # need to give number of cameras in the solution

# visualise_objective_function(best_solution, instance_size=instance_size, nb_cameras=actual_cameras, instance_file=myinst)

In [ ]:
hof[0], num_cells

In [ ]:
best_solution=np.zeros(num_cells)
best_solution=convert_individual_bitstring(hof[0])
actual_cameras=sum(best_solution)  # need to give number of cameras in the solution
visualise_objective_function(best_solution, instance_size=instance_size, nb_cameras=actual_cameras, instance_file=myinst)

In [ ]:
print("max fitness found is %s at gen 5" % (best))